# Data Driven Modeling 3: Random Forest Regression

In [ ]:
#%% Import necessary packages
import numpy as np
import pandas as pd
import matplotlib
import time
import matplotlib.patches as mpatches
import warnings
from sklearn.exceptions import UndefinedMetricWarning
from matplotlib import pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn import linear_model
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold, cross_validate, LeaveOneOut, train_test_split
from sklearn.ensemble import RandomForestRegressor 


# Set matplotlib parameters for plotting
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['axes.linewidth'] = 1.5
matplotlib.rcParams['xtick.major.size'] = 8
matplotlib.rcParams['xtick.major.width'] = 2
matplotlib.rcParams['ytick.major.size'] = 8
matplotlib.rcParams['ytick.major.width'] = 2
matplotlib.rcParams['figure.dpi'] = 100.

## 1. Inspect the dataset

In [ ]:
# fetch the housing dataset
dataset = fetch_california_housing()

# extract the features X and reponse y 
X_full, y_full = dataset.data, dataset.target
# check dataset sizes
print(X_full.shape)

# print out the feature names
X_names = dataset.feature_names
print(X_names)
y_name = 'Housing Price'

# Take only 2 features to make visualization easier
# Feature 0 and feature 1 have very different scales and distributions 
fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(X_full[:,0], X_full[:,2], s = 30, alpha = 0.5)
ax.set_xlabel(X_names[0])
ax.set_ylabel(X_names[1])

# plot the last two features which gives the shape of the state of California! 
fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(X_full[:,-1], X_full[:,-2], s = 30, alpha = 0.5)
ax.set_xlabel(X_names[-1])
ax.set_ylabel(X_names[-2])

## 2. Standardize the data to a zero mean and unit variance

In [ ]:
# Preprocess the data 
# Standardize 
scaler = StandardScaler().fit(X_full)
Xs = scaler.transform(X_full)

# Export the mean and std of the original data  
X_std = scaler.scale_ # std for each x variable
print('The std of each column in original X:')
print(X_std)

X_mean = scaler.mean_ # mean for each x variable
print('The std of each column in original X:')
print(X_std)

# Check if there have a unit variance 
Xs_std = np.std(Xs, axis = 0) 
print('The std of each column in standardized X:')
print(Xs_std)

Xs_mean = np.mean(Xs, axis = 0)
print('The mean of each column standardized X:')
print(Xs_mean)

# Feature 0 and feature 1 after standardization
fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(Xs[:,0], Xs[:,2], s = 30, alpha = 0.5)
ax.set_xlabel(X_names[0])
ax.set_ylabel(X_names[1])

# Last two features after standardization
fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(Xs[:,-1], Xs[:,-2], s = 30, alpha = 0.5)
ax.set_xlabel(X_names[-1])
ax.set_ylabel(X_names[-2])

# Assign Xs to X
X = Xs.copy()
y = y_full.copy()

## 3. Split the data into training and test set, set up cross-validation

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# check train and test set sizes
print(X_train.shape)
print(X_test.shape)
                               
loo = LeaveOneOut()

## 4. Create helper functions

In [ ]:
def cross_validation(X, y, estimator): 
    '''
    Cross-validation
    '''
    loo = LeaveOneOut()
    scores  = cross_validate(estimator, X, y, cv = loo,
                                scoring=('neg_mean_squared_error', 'r2'),
                                return_train_score=True)
    # RMSE for repeated 10 fold test data 
    test_RMSE = np.sqrt(np.abs(scores['test_neg_mean_squared_error'])) 
    test_RMSE_mean = np.mean(test_RMSE)
    test_RMSE_std = np.std(test_RMSE)
    
    train_r2 = scores['train_r2'] 
    train_r2_mean =  np.mean(train_r2)
    train_r2_std = np.std(train_r2)
    
    return [test_RMSE_mean, test_RMSE_std, train_r2_mean, train_r2_std]


## 5. Find the optimal number of estimators

In [ ]:
# Turn off warnings for calculating r2 of a single point
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

# Train the random forest model using Grid search
train_flag = True

if train_flag:
    # grid search for n_estimator. Can change it based on your own system.
    n_estimators_grid = range(1,51, 1)
    test_RMSE_m = np.zeros(len(n_estimators_grid)) 
    progress = 0
    
    start_time = time.time()
    for i, n_estimators_i in enumerate(n_estimators_grid):
        progress += 1
        per = progress/len(n_estimators_grid)*100
        print('Training {0:.5f} % Done!'.format(per))
            
        rf = RandomForestRegressor(n_estimators = n_estimators_i,random_state=0)
                                                                            
        [test_RMSE_mean, test_RMSE_std, train_r2_mean, train_r2_std] = cross_validation(X_train, y_train, rf)
        test_RMSE_m[i] = test_RMSE_mean
    
    end_time = time.time()
    
    print("Tuning hyperparameter takes {0:.2f} minutes".format((end_time-start_time)/60.0))
    
    # Final the optimal model
    test_RMSE_m = np.around(test_RMSE_m, decimals = 3)
    opt_ind = np.unravel_index(np.argmin(test_RMSE_m, axis=None), test_RMSE_m.shape)
    n_estimators_opt = n_estimators_grid[opt_ind[0]]

else: 
    n_estimators_opt = 50

print('The optimal number of estimator is: {}'.format(n_estimators_opt))


In [ ]:
#%% Fit the model using the optimal estimator    
rf_opt = RandomForestRegressor(n_estimators = n_estimators_opt, random_state=0)                                     
rf_opt.fit(X_train, y_train)

# Calculated the error on test data
y_predict_test = rf_opt.predict(X_test)
RMSE_test = np.sqrt(mean_squared_error(y_test, y_predict_test))

print('RMSE of test set is: {}'.format(RMSE_test))